In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import acquire as a
import env

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

In [2]:
# Import .txt file and convert it to a DataFrame object
df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [3]:
def acquire_curriculum_data():
    #Get the data from text file on local repo
    df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])
    return df

In [4]:
df = acquire_curriculum_data()

In [5]:
df.head()

,date,time,page,id,cohort,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [14]:
lessons = df.page

In [11]:
lessons.unique()

array(['/', 'java-ii', 'java-ii/object-oriented-programming', ...,
       'florence-python-assessment.html', 'javascript-i/dom',
       'appendix/professional-development/post-interview-review-form'],
      dtype=object)

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [24]:
#Total Users
df.id.nunique()

981

In [27]:
df.id.value_counts().tail(20)

348    3
176    3
177    3
852    2
97     2
573    2
574    2
649    1
940    1
165    1
879    1
652    1
592    1
619    1
593    1
66     1
163    1
918    1
212    1
952    1
Name: id, dtype: int64

In [20]:
df.ip.nunique()

5531

In [25]:
df.ip.value_counts()

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64

In [29]:
df.cohort.value_counts()

28.0     84031
33.0     40730
29.0     38096
62.0     37109
53.0     36902
24.0     35636
57.0     33844
56.0     33568
51.0     32888
59.0     32015
22.0     30926
58.0     29855
32.0     29356
23.0     28534
52.0     28033
26.0     27749
34.0     26538
25.0     25586
31.0     25359
132.0    23691
55.0     21582
27.0     20743
61.0     17713
134.0    16623
135.0    16397
133.0    14715
14.0      9587
1.0       8890
137.0     8562
21.0      7444
138.0     7276
17.0      4954
13.0      2845
18.0      2158
8.0       1712
139.0     1672
19.0      1237
16.0       755
15.0       691
7.0        598
12.0       302
11.0       253
2.0         93
6.0         72
9.0          5
4.0          4
5.0          1
Name: cohort, dtype: int64

In [32]:
df.isnull().sum()

date          0
time          0
page          1
id            0
cohort    52893
ip            0
dtype: int64

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    900223 non-null  object 
 1   time    900223 non-null  object 
 2   page    900222 non-null  object 
 3   id      900223 non-null  int64  
 4   cohort  847330 non-null  float64
 5   ip      900223 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 41.2+ MB


In [2]:
def acquire_cohort_logs(user=env.user, password=env.password, host=env.host):
    '''
    This function queries the Codeup MySQL curriculum_logs database and returns a dataframe
    '''
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
    query = '''
    SELECT date, path as endpoint, user_id, cohort_id, name, start_date, end_date, ip as source_ip, program_id
        FROM logs l
        LEFT JOIN cohorts c ON l.cohort_id = c.id;
    '''
    df = pd.read_sql(query, url)
    #Save file to csv
    df.to_csv('cohort_logs.csv')
    return df

In [3]:
df = acquire_cohort_logs(user=env.user, password=env.password, host=env.host)
df.head()

KeyboardInterrupt: 

In [38]:
def clean_cohort_logs(df):
    #Get csv from file
    df = pd.read_csv('cohort_logs.csv', index_col=[0])
    #Changing date, start and end dates to datetime fields
    df["date"]= pd.to_datetime(df["date"])
    df["start_date"]= pd.to_datetime(df["start_date"])
    df["end_date"]= pd.to_datetime(df["end_date"])
    #Setting index as date time
    #Set datetime index
    df = df.set_index(df.date)
    #Creating new column for program length
    df['program_length'] = df.end_date - df.start_date
    #Create column with program name
    #Change data types as needed
    df["program_id"]= df["program_id"].astype(str)
    #Copying these values to a new column program name
    df['program_name'] = df['program_id']
    #Mapping the real names of the program
    df["program_name"] = df["program_name"].map({'1.0':'full stack PHP','2.0':'full stack Java','3.0':'data science','4.0':'front end'})
    
    return df

In [39]:
df = clean_cohort_logs(df)
df.head()

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length,program_name
date,,,,,,,,,,,
2018-01-26,2018-01-26,/,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,java-ii,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,java-ii/object-oriented-programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,slides/object_oriented_programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,javascript-i/conditionals,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java


In [45]:
#Taking all activity for students, not staff
student_df = df[df.name != "Staff"]
student_df.head()

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length,program_name
date,,,,,,,,,,,
2018-01-26,2018-01-26,/,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,java-ii,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,java-ii/object-oriented-programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,slides/object_oriented_programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2018-01-26,2018-01-26,javascript-i/conditionals,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java


In [44]:
#Taking all activity from students active in course(from start date to end date)
active_df = student_df[(student_df.end_date >= student_df.index) & (student_df.start_date <= student_df.index)]
active_df = active_df[active_df['endpoint'] != '/']
active_df.head()

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length,program_name
date,,,,,,,,,,,
2018-01-26,2018-01-26,javascript-i/conditionals,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java
2018-01-26,2018-01-26,javascript-i/loops,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java
2018-01-26,2018-01-26,javascript-i/conditionals,3,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java
2018-01-26,2018-01-26,javascript-i/functions,3,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java
2018-01-26,2018-01-26,javascript-i/loops,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java


In [52]:
active_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 617166 entries, 2018-01-26 to 2021-04-21
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   date            617166 non-null  datetime64[ns] 
 1   endpoint        617165 non-null  object         
 2   user_id         617166 non-null  int64          
 3   cohort_id       617166 non-null  float64        
 4   name            617166 non-null  object         
 5   start_date      617166 non-null  datetime64[ns] 
 6   end_date        617166 non-null  datetime64[ns] 
 7   source_ip       617166 non-null  object         
 8   program_id      617166 non-null  object         
 9   program_length  617166 non-null  timedelta64[ns]
 10  program_name    617166 non-null  object         
dtypes: datetime64[ns](3), float64(1), int64(1), object(5), timedelta64[ns](1)
memory usage: 56.5+ MB


In [47]:
#Aggregating pages by student
pages_by_student = active_df.groupby(['user_id'])['endpoint'].value_counts()
pages_by_student

user_id  endpoint                                        
2        java-iii                                            50
         mysql                                               32
         java-ii                                             30
         spring                                              29
         jquery                                              28
                                                             ..
981      examples/css/bootstrap-grids.html                    3
         appendix/professional-development/t-block-resume     1
         examples/bootstrap-grid/assets.zip                   1
         html-css/css-ii                                      1
         javascript-ii                                        1
Name: endpoint, Length: 88202, dtype: int64

In [49]:
#make a df
pages_by_student =pd.DataFrame(pages_by_student)
pages_by_student.head()

endpoint
user_id endpoint          
2       java-iii        50
        mysql           32
        java-ii         30
        spring          29
        jquery          28

In [53]:
pages_by_student.columns = ['endpoint_count']
pages_by_student = pages_by_student.reset_index()
pages_by_student = pages_by_student.groupby('user_id').sum('endpoint_count')
pages_by_student.nsmallest(n=5, columns='endpoint_count')

,endpoint_vc
user_id,
832,2
278,3
539,4
956,4
388,7
